In [ ]:
import cv2
import numpy as np
import pyrealsense2 as rs
import matplotlib.pyplot as plt

# Load image for lane detection
image = cv2.imread('lanes.png')
lane_image = np.copy(image)
gray = cv2.cvtColor(lane_image, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (9, 9), 0)
canny = cv2.Canny(blur, 50, 100)

def region_of_interest(image):
    height = image.shape[0]
    polygons = np.array([
        [(1000, height), (1700, 1800), (2400, 1800), (3500, height)]
    ])
    mask = np.zeros_like(image)
    cv2.fillPoly(mask, polygons, 255)
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image

# Apply region of interest
cropped_image = region_of_interest(canny)

# Function to display lines
def display_lines(image, lines):
    line_image = np.zeros_like(image)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line.reshape(4)
            cv2.line(line_image, (x1, y1), (x2, y2), (255, 0, 0), 10)
    return line_image

# Detect lines
lines = cv2.HoughLinesP(cropped_image, 2, np.pi / 180, 100, np.array([]), minLineLength=5, maxLineGap=70)
line_image = display_lines(lane_image, lines)
combo_image = cv2.addWeighted(lane_image, 0.6, line_image, 1, 1)

# Function to get coordinates
def make_coordinates(image, line_parameters):
    slope, intercept = line_parameters
    y1 = image.shape[0]
    y2 = int(y1 * (3 / 5))
    x1 = int((y1 - intercept) / slope)
    x2 = int((y2 - intercept) / slope)
    return np.array([x1, y1, x2, y2])

# Average slope intercept function
def average_slope_intercept(image, lines):
    left_fit = []
    right_fit = []
    for line in lines:
        x1, y1, x2, y2 = line.reshape(4)
        parameters = np.polyfit((x1, x2), (y1, y2), 1)
        slope = parameters[0]
        intercept = parameters[1]
        if slope < 0:
            left_fit.append((slope, intercept))
        else:
            right_fit.append((slope, intercept))

    if len(left_fit) > 0:
        left_fit_average = np.average(left_fit, axis=0)
        left_line = make_coordinates(image, left_fit_average)
    else:
        left_line = np.array([0, 0, 0, 0])

    if len(right_fit) > 0:
        right_fit_average = np.average(right_fit, axis=0)
        right_line = make_coordinates(image, right_fit_average)
    else:
        right_line = np.array([0, 0, 0, 0])

    return np.array([left_line, right_line])

# Find average lines
averaged_lines = average_slope_intercept(lane_image, lines)
line_image = display_lines(lane_image, averaged_lines)

combo_image2 = cv2.addWeighted(lane_image, 0.4, line_image, 1, 1)

# Setup RealSense camera
class DepthCamera:
    def __init__(self):
        self.pipeline = rs.pipeline()
        config = rs.config()
        config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
        self.pipeline.start(config)

    def get_frame(self):
        frames = self.pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        if not depth_frame:
            return False, None
        depth_image = np.asanyarray(depth_frame.get_data())
        return True, depth_image

    def release(self):
        self.pipeline.stop()

# Initialize the depth camera
dc = DepthCamera()

# Main loop to get frames and detect distances
while True:
    ret, depth_frame = dc.get_frame()
    if not ret:
        continue

    # Coordinates of detected points (x1, y1, x2, y2)
    (x1, y1), (x2, y2) = averaged_lines[0][:2], averaged_lines[1][:2]

    # Scale these coordinates to the depth camera resolution (640x480)
    original_height, original_width = lane_image.shape[:2]
    scale_x = 640 / original_width
    scale_y = 480 / original_height

    # Apply scaling to detected points
    x1_scale = int(x1 * scale_x)
    y1_scale = int(y1 * scale_y)
    x2_scale = int(x2 * scale_x)
    y2_scale = int(y2 * scale_y)

    # Get the depth values for the scaled coordinates
    if 0 <= x1_scale < 640 and 0 <= y1_scale < 480:
        distance_1 = depth_frame[y1_scale, x1_scale]
    else:
        distance_1 = None

    if 0 <= x2_scale < 640 and 0 <= y2_scale < 480:
        distance_2 = depth_frame[y2_scale, x2_scale]
    else:
        distance_2 = None

    # Print the distances
    print(f"Distance 1: {distance_1} mm, Distance 2: {distance_2} mm")

    key = cv2.waitKey(1)
    if key == 27:  # Press 'ESC' to exit
        break

# Release the camera
dc.release()
cv2.destroyAllWindows()